<a href="https://colab.research.google.com/github/catundchat/Summary4dialogue/blob/main/transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece
!nvidia-smi

In [ ]:
import transformers
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/mt5-large")

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-large")

# Get the conversation that you want to summarize
conversation = '''
情绪问题

好的，你最近有什么情绪问题吗？比如说，你是否经常感到沮丧、焦虑或者压力很大？

控制不住自己情绪怎么办

我理解你的感受。控制不住自己的情绪可能会让人感到无助和困惑。我想问一下，你最近有遇到什么事情让你感到情绪失控吗？

失恋

失恋是一件很痛苦的事情，我能理解你现在的情绪。你可以告诉我更多关于你失恋的情况吗？比如说，你们分手的原因是什么？你现在的情绪是什么样的？
'''

# Convert the conversation to tensor using tokenizer
input_ids = tokenizer.encode(conversation, return_tensors='pt')

# Summarize the conversation
if len(conversation)<100:
  summary_length = len(conversation)//2.5
  summary = model.generate(
    input_ids, 
    max_length=summary_length, 
    num_return_sequences=1, 
    length_penalty=1.25, 
    no_repeat_ngram_size=2,
    temperature = 1.0,
    top_k = 30
    )
elif 100<=len(conversation)<500:
  summary_length = len(conversation)//3.33
  summary = model.generate(
    input_ids, 
    max_length=summary_length, 
    num_return_sequences=1, 
    length_penalty=1.25, 
    no_repeat_ngram_size=2,
    temperature = 1.0,
    top_k = 35
    )
elif 500<=len(conversation)<1000:
  summary_length = len(conversation)//4
  summary = model.generate(
    input_ids, 
    max_length=summary_length, 
    num_return_sequences=1, 
    length_penalty=1.5, 
    no_repeat_ngram_size=2,
    temperature = 1.0,
    top_k = 60
    )
else:
  summary_length = len(conversation)//4
  summary = model.generate(
    input_ids, 
    max_length=summary_length, 
    num_return_sequences=1, 
    length_penalty=1.5, 
    no_repeat_ngram_size=2,
    temperature = 1.0,
    top_k = 100
    )

# Decode the summary using tokenizer and remove special tokens
summary_text = tokenizer.decode(summary[0], skip_special_tokens=True)
summary_text = re.sub("extra_id_[0-9]+", "", summary_text).strip()

# print(summary_text)
# extract content between two <>
# result = summary_text.split("<>")[1].split("<")[0]
result = summary_text.split("<>")[1].split("<")[0].strip().replace(",", "").replace(" ", "")

# output
print(result)